In [97]:
import pymysql
import pandas as pd
import numpy as np

In [98]:
df = pd.read_csv('migracion_relacional.csv')

In [99]:
df.rename(columns={'Country of birth/nationality': 'CountryOfBirth'}, inplace=True)

In [100]:
df.rename(columns={'aux': 'Id_pais'}, inplace=True)

In [101]:
df.head()

,CountryOfBirth,Country,Year,value,Id_pais
0,India,Australia,2012,27830.0,2012India
1,India,Canada,2012,30915.0,2012India
2,India,Germany,2012,18063.0,2012India
3,India,New Zealand,2012,8564.0,2012India
4,India,United States,2012,64694.0,2012India


In [102]:
df['CountryOfBirth'] = df['CountryOfBirth'].astype(str)

In [103]:
df['Country'] = df['Country'].astype(str)

In [104]:
df['Id_pais'] = df['Id_pais'].astype(str)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CountryOfBirth  225 non-null    object 
 1   Country         225 non-null    object 
 2   Year            225 non-null    int64  
 3   value           225 non-null    float64
 4   Id_pais         225 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 8.9+ KB


In [106]:
#Establezco conexion
conexion = pymysql.connect(
    host = "database-migraciones.c5vthlbc6wtm.us-east-1.rds.amazonaws.com",
    user= "***********",
    passwd = "***********",
    db = "migraciones"

)

In [107]:
cursor = conexion.cursor()

In [108]:
cursor.execute("""DROP TABLE IF EXISTS migracion_relacional""")

0

In [109]:
cursor.execute("""CREATE TABLE migracion_relacional (
    Id INT AUTO_INCREMENT PRIMARY KEY,
    Id_pais VARCHAR(45),
    CountryOfBirth VARCHAR(45),
    Country VARCHAR(45),
    Year INT,
    value FLOAT
)""")

0

In [110]:
sp = df.shape

In [111]:
len = sp[0]

In [112]:
df_lista = []
for i in range (0,len):
    df_lista.append((df.CountryOfBirth[i], df.Country[i],df.Year[i], df.value[i],df.Id_pais[i])) 

In [113]:
cursor.executemany("""insert into migracion_relacional(CountryOfBirth, Country, Year, value, Id_pais)
                    values (%s, %s, %s, %s, %s)""", df_lista
)

225

In [114]:
conexion.commit()

In [115]:
cursor.execute("""ALTER TABLE migracion_relacional 
                    ADD CONSTRAINT migracion_relacional_fk_macroeconomico FOREIGN KEY (Id_pais) REFERENCES macroeconomico(Id)""")

225

In [116]:
conexion.close()